# Extracción y transformación de datos sobre contaminación sonora.

Fuente de información: https://noiseproject.org/data-download/

In [60]:
# Importo librerias necesarias
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
from pandas.errors import EmptyDataError
import warnings

warnings.filterwarnings('ignore')

In [61]:
df = ''
while not isinstance(df, pd.core.frame.DataFrame):
  try:
    csv_url = 'https://citsci-noise-server.ornith.cornell.edu/noise/download/csv?lang=en'
    response_csv = requests.get(csv_url)
    csv_content = io.StringIO(response_csv.text)
    df = pd.read_csv(csv_content)
    break
  except EmptyDataError:
    continue

df

,Start date/time in UTC,Timezone,Duration (s),Observer ID,Latitude,Longitude,Mean volume (dBA),Mood,Could control noise exposure,Indoors,Emoji,Emoji descriptions,Nominated as noise refuge
0,2022-02-08 14:07:20,America/New_York,29,56,42.71516,-76.40451,42,2,False,True,🗣🤣,"speaking head in silhouette,rolling on the flo...",False
1,2022-02-08 14:56:00,America/New_York,29,54,42.71489,-76.40509,39,1,True,True,🎶,multiple musical notes,False
2,2022-02-08 15:46:02,America/New_York,29,55,42.38951,-71.14181,70,3,False,False,🚗🛣,"automobile,motorway",False
3,2022-02-09 10:17:05,America/New_York,29,61,41.82063,-71.42895,72,1,True,True,🎶🎻🎺,"multiple musical notes,violin,trumpet",False
4,2022-02-09 11:07:23,America/New_York,29,64,42.47998,-76.45123,39,2,False,True,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,2023-08-18 03:10:39,Australia/Brisbane,29,846,-27.09049,152.92533,70,4,False,False,NaN,NaN,False
1662,2023-08-18 03:12:10,Australia/Brisbane,29,846,-27.09057,152.92541,68,4,False,False,NaN,NaN,False
1663,2023-08-18 03:13:55,Australia/Brisbane,29,846,-27.09058,152.92542,75,4,False,False,NaN,NaN,False
1664,2023-08-18 03:14:45,Australia/Brisbane,29,846,-27.09051,152.92543,74,5,False,False,NaN,NaN,False


In [63]:
# Filtro la data por las coordenadas de Manhattan
df = df[(df.Latitude.between(40.698470, 40.878669)) & (df.Longitude.between(-74.019100, -73.903648))]

# Convierto las columnas de fechas y tiempo a columnas tipo datetime
df.loc[:, 'Start date/time in UTC'] = pd.to_datetime(df['Start date/time in UTC'])
df['Start date UTC'] = df['Start date/time in UTC'].dt.normalize()
df['Start time UTC'] = df['Start date/time in UTC'].dt.time

df['Timezone'] = df['Timezone'].str.replace('America/New_York', 'New York')
df['City'] = 'Manhattan'
df = df.rename(columns={'Emoji descriptions':'Noise Source'})

emotions_dict = {
    1: "Very happy",
    2: "Happy",
    3: "Neutral",
    4: "Unhappy",
    5: "Very unhappy"
}

df['Mood'] = df['Mood'].map(emotions_dict)

# Selecciono las columnas de valor
df = df[['Start date/time in UTC',
         'Start date UTC',
         'Start time UTC',
         'Timezone',
         'City',
         'Latitude',
         'Longitude',
         'Duration (s)',
         'Mean volume (dBA)',
         'Mood',
         'Could control noise exposure',
         'Indoors',
         'Noise Source',
         'Nominated as noise refuge']]

# Selecciono solamente los registros provenientes de ruidos de vehiculos y/o de taxis
df = df[(df['Noise Source'].str.contains('taxi')) | (df['Noise Source'].str.contains('automobile'))]

df = df.sort_values(by='Start date/time in UTC', ascending=False).reset_index(drop=True)
df

,Start date/time in UTC,Start date UTC,Start time UTC,Timezone,City,Latitude,Longitude,Duration (s),Mean volume (dBA),Mood,Could control noise exposure,Indoors,Noise Source,Nominated as noise refuge
0,2023-07-14 18:16:50,2023-07-14,18:16:50,New York,Manhattan,40.78362,-73.97919,29,69,Neutral,False,False,"taxi,automobile,delivery truck,bus",False
1,2023-07-13 20:18:45,2023-07-13,20:18:45,New York,Manhattan,40.78404,-73.95891,29,65,Happy,False,False,"airplane,helicopter,taxi,automobile,speaking h...",False
2,2023-07-13 20:14:52,2023-07-13,20:14:52,New York,Manhattan,40.78633,-73.95767,29,68,Neutral,False,False,"helicopter,automobile,speaking head in silhoue...",False
3,2023-07-08 16:40:51,2023-07-08,16:40:51,New York,Manhattan,40.78705,-73.95734,29,64,Neutral,False,False,"automobile,bus,delivery truck,airplane,rolling...",True
4,2023-07-08 16:26:08,2023-07-08,16:26:08,New York,Manhattan,40.79337,-73.96617,29,68,Neutral,False,False,"helicopter,bus,automobile,taxi,delivery truck,...",False
5,2023-07-08 16:23:46,2023-07-08,16:23:46,New York,Manhattan,40.79367,-73.96727,29,78,Unhappy,False,False,"delivery truck,taxi,automobile,bus,speaking he...",False
6,2022-09-11 09:31:56,2022-09-11,09:31:56,New York,Manhattan,40.86174,-73.92150,29,68,Very unhappy,False,True,"automobile,delivery truck",True
7,2022-08-17 18:35:43,2022-08-17,18:35:43,New York,Manhattan,40.82965,-73.94658,29,76,Very unhappy,False,False,"automobile,multiple musical notes",False
8,2022-07-04 23:58:52,2022-07-04,23:58:52,New York,Manhattan,40.85693,-73.92778,29,78,Very unhappy,False,True,"racing motorcycle,taxi,speaker with three soun...",True
9,2022-05-14 13:02:23,2022-05-14,13:02:23,New York,Manhattan,40.78601,-73.97536,29,68,Unhappy,False,False,"taxi,ambulance",NaN
